In [3]:
pip install selenium pandas requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Joshk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

#책 소개에서 HTML 태그 제거 함수
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text

# 이미지 다운로드 함수
def download_image(image_url, save_path):
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)

# 디렉토리 생성
save_dir = 'book_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 웹 드라이버 설정
driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/SteadySeller?categoryNumber=001')
time.sleep(2)

# 로그인 작업
sign = driver.find_element(By.XPATH, '//*[@id="LoginText"]/a')
driver.execute_script("arguments[0].click();", sign)
time.sleep(2)

# ID 입력
id = driver.find_element(By.XPATH, "//*[@id='SMemberID']")
id.click()
id.send_keys('예스24 아이디')

# 패스워드 입력
password = driver.find_element(By.XPATH, "//*[@id='SMemberPassword']")
password.click()
password.send_keys('예스24 비밀번호')

# 로그인 버튼 클릭
button = driver.find_element(By.XPATH, '//*[@id="btnLogin"]/span')
button.click()

try:
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="pg_size"]'))  # 'pg_size'는 실제 select 태그의 id 혹은 적절한 XPATH로 교체
    )
    select = Select(select_element)
    select.select_by_value('120')
except Exception as e:
    print(f"Error selecting page size: {e}")
    driver.quit()

book = []

# 최신간 20개 클릭하고 내부 값 가져오는 작업
for page in range(1, 121):
    elements = driver.find_element(By.XPATH, f"//*[@id='yesBestList']/li[{page}]/div/div[1]/div[1]/span/span/a/em/img")
    driver.execute_script("arguments[0].click();", elements)
    time.sleep(2)
    
    img = driver.find_element(By.CLASS_NAME, "gImg")
    src = img.get_attribute("src")
    
    # 이미지 다운로드 및 로컬 저장
    image_name = f"{src.split('/')[-2]}.jpg"
    image_path = os.path.join(save_dir, image_name)
    download_image(src, image_path)
    
    제목 = driver.find_element(By.XPATH, '//*[@id="yDetailTopWrap"]/div[2]/div[1]/div/h2').text
    저자 = driver.find_element(By.XPATH, '//*[@id="yDetailTopWrap"]/div[2]/div[1]/span[2]/span[1]/a[1]').text
    출판사 = driver.find_element(By.XPATH, '//*[@id="yDetailTopWrap"]/div[2]/div[1]/span[2]/span[2]/a').text
    발행일 = driver.find_element(By.XPATH, '//*[@id="yDetailTopWrap"]/div[2]/div[1]/span[2]/span[3]').text
    ISBN = driver.find_element(By.XPATH, '//*[@id="infoset_specific"]/div[2]/div/table/tbody/tr[3]/td').text.strip()
    ISBN = str(ISBN)
    쪽수 = driver.find_element(By.XPATH, '//*[@id="infoset_specific"]/div[2]/div/table/tbody/tr[2]/td').text
    쪽수 = 쪽수[:3].strip()
    장르 = driver.find_element(By.XPATH, '//*[@id="yLocation"]/div/div[2]/a').text
    책소개_raw = driver.find_element(By.CLASS_NAME, 'txtContentText').get_attribute('value')
    책소개 = remove_html_tags(책소개_raw)

    book.append({
        "title": 제목,
        "author": 저자,
        "publisher": 출판사,
        "publistDate": 발행일,
        "ISBN": ISBN,
        "pages": 쪽수,
        "genre": 장르,
        "description": 책소개,
        "book_img_url": image_name,
    })
    
    driver.back()
    time.sleep(2)
    
df = pd.DataFrame(book)

# 실행 후 ISBN 서식 변경 필요
df.to_csv("book.csv", encoding="utf-8-sig")

driver.quit()
